# 1.1 Clean and Examine
This notebook contains the process used to import, examine, and clean the dataset containing COVID hospital cases by county. It also includes the process used to slim down the US gov's USA county shapefiles to only include the contiguous US states.

### 1.1.1 Import Packages

In [105]:
import numpy as np #linear algebra
import pandas as pd #data processing, CSV file I/O
import matplotlib.pyplot as plt
import seaborn as sns

# Import libraries for reading geographic data
import geopandas as gpd
from shapely.geometry import Point, shape
import requests
import zipfile
import io
import os

%matplotlib inline

In [106]:
os.getcwd()

'/Users/tiffanyflor/Dropbox/MyProjects/usacovidhosp/notebooks'

In [107]:
# List files in the raw data directory

for dirname, _, filenames in os.walk('../data/raw/'):
    for filename in filenames:
        print(filename)

tl_2017_us_county.cpg
tl_2017_us_county.shp
Merged_Final.csv
tl_2017_us_county.shx
.gitkeep
tl_2017_us_county.dbf
tl_2017_us_county.prj


### 1.1.2 Examine Data Set
Import dataset as a pandas DataFrame. Examine metadata and univariate descriptive statistics.

In [108]:
# Merged_Final.csv --> contains USA hospitals with COVID by county
# Put into dataframe
df = pd.read_csv('../data/raw/Merged_Final.csv')

In [109]:
# View data
df.head()

,County Name,Facility_Name,Full_Address,State,Total,latitudes,longitudes
0,HOUSTON,SOUTHEAST ALABAMA MEDICAL CENTER,"1108 ROSS CLARK CIRCLE,DOTHAN,HOUSTON,AL",AL,2427,31.215827,-85.363433
1,MARSHALL,MARSHALL MEDICAL CENTERS,"2505 U S HIGHWAY 431 NORTH,BOAZ,MARSHALL,AL",AL,5104,34.601233,-87.123343
2,LAUDERDALE,NORTH ALABAMA MEDICAL CENTER,"1701 VETERANS DRIVE,FLORENCE,LAUDERDALE,AL",AL,2120,34.805043,-87.650814
3,JEFFERSON,ST. VINCENT'S EAST,"50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,JEFFERSO...",AL,26142,33.564772,-86.760155
4,SHELBY,SHELBY BAPTIST MEDICAL CENTER,"1000 FIRST STREET NORTH,ALABASTER,SHELBY,AL",AL,20720,33.075378,-86.519100


In [110]:
# Get rows and columns
df.shape

(4478, 7)

In [111]:
# List data types and counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4478 entries, 0 to 4477
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   County Name    4478 non-null   object 
 1   Facility_Name  4478 non-null   object 
 2   Full_Address   4478 non-null   object 
 3   State          4478 non-null   object 
 4   Total          4478 non-null   int64  
 5   latitudes      4478 non-null   float64
 6   longitudes     4478 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 245.0+ KB


In [112]:
# Summary statistics for numerical columns
df.describe(include=[np.number])

,Total,latitudes,longitudes
count,4478.000000,4478.000000,4478.000000
mean,13635.779812,37.942630,-93.095641
std,27010.816470,5.383805,14.995810
min,0.000000,18.372500,-159.671259
25%,497.000000,34.001714,-99.310467
50%,3581.000000,38.773624,-90.291356
75%,14835.000000,41.682943,-82.382111
max,159045.000000,58.329259,-66.046000


In [113]:
# Summary statistics for object and categorical columns
df.describe(include=[np.object, pd.Categorical])

,County Name,Facility_Name,Full_Address,State
count,4478,4478,4478,4478
unique,1273,4341,4476,51
top,LOS ANGELES,MEMORIAL HOSPITAL,"47 SOUTH FOURTH ST,ROLLING FORK,SHARKEY,MS",TX
freq,90,12,2,407


### 1.1.3 Clean Data
Drop unnecessary rows and remove duplicates. Reset index for a clean dataframe.

In [114]:
# Check out the number of states listed
df['State'].nunique()

51

In [115]:
# Notice it includes Puerto Rico
df['State'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
       'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
       'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
       'ND', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [116]:
# Since we are only concerned with the 50 US states, we'll drop rows containing PR
df = df[df.State != 'PR']
df.shape

(4460, 7)

In [117]:
# Check for duplicate hospitals
df['Full_Address'].value_counts()

101 CIRCLE DRIVE,HILLSBORO,HILL,TX                 2
47 SOUTH FOURTH ST,ROLLING FORK,SHARKEY,MS         2
1500 S MAIN ST,FORT WORTH,TARRANT,TX               1
751 NE BLAKELY DR,ISSAQUAH,KING,WA                 1
3001 BROADMOOR BLVD NE,RIO RANCHO,SANDOVAL,NM      1
                                                  ..
172 KINSLEY ST,NASHUA,HILLSBOROUGH,NH              1
501 NORTH LANSDOWNE AVE,DREXEL HILL,DELAWARE,PA    1
1501 SOUTH DIXIE STREET,HORSE CAVE,HART,KY         1
41 EAST POST R0AD,WHITE PLAINS,WESTCHESTER,NY      1
9191 GRANT ST,THORNTON,ADAMS,CO                    1
Name: Full_Address, Length: 4458, dtype: int64

In [118]:
# Examine hospitals with more than one listing
df[df['Full_Address'] == '47 SOUTH FOURTH ST,ROLLING FORK,SHARKEY,MS']

,County Name,Facility_Name,Full_Address,State,Total,latitudes,longitudes
2158,SHARKEY,SHARKEY ISSAQUENA COMMUNITY HOSPITAL,"47 SOUTH FOURTH ST,ROLLING FORK,SHARKEY,MS",MS,65,34.004727,-90.070306
2212,SHARKEY,SHARKEY ISSAQUENA COMMUNITY HOSPITAL,"47 SOUTH FOURTH ST,ROLLING FORK,SHARKEY,MS",MS,65,34.004727,-90.070306


In [119]:
df[df['Full_Address'] == '101 CIRCLE DRIVE,HILLSBORO,HILL,TX']

,County Name,Facility_Name,Full_Address,State,Total,latitudes,longitudes
3725,HILL,HILL REGIONAL HOSPITAL,"101 CIRCLE DRIVE,HILLSBORO,HILL,TX",TX,239,31.269893,-98.943568
3947,HILL,HILL REGIONAL HOSPITAL,"101 CIRCLE DRIVE,HILLSBORO,HILL,TX",TX,239,31.269893,-98.943568


In [120]:
# Review areas around index to determine if there is an obvious order to hospital input
df.iloc[2155:2163]

,County Name,Facility_Name,Full_Address,State,Total,latitudes,longitudes
2155,HINDS,MERIT HEALTH CENTRAL,"1850 CHADWICK DR,JACKSON,HINDS,MS",MS,3691,32.287775,-90.254940
2156,WAYNE,WAYNE GENERAL HOSPITAL,"950 MATTHEW DR,WAYNESBORO,WAYNE,MS",MS,29753,31.673138,-88.634152
2157,FORREST,FORREST GENERAL HOSPITAL,"6051 US HIGHWAY 49,HATTIESBURG,FORREST,MS",MS,1197,33.079453,-89.858575
2158,SHARKEY,SHARKEY ISSAQUENA COMMUNITY HOSPITAL,"47 SOUTH FOURTH ST,ROLLING FORK,SHARKEY,MS",MS,65,34.004727,-90.070306
2159,LAUDERDALE,ANDERSON REGIONAL MEDICAL CENTER SOUTH CAMPUS,"1102 CONSTITUTION AVENUE,MERIDIAN,LAUDERDALE,MS",MS,2120,31.686577,-88.652958
2160,WASHINGTON,DELTA REGIONAL MEDICAL CENTER,"1400 E UNION ST,GREENVILLE,WASHINGTON,MS",MS,17929,33.410527,-91.035356
2161,ADAMS,MERIT HEALTH NATCHEZ,"52 SERGEANT PRENTISS DRIVE,NATCHEZ,ADAMS,MS",MS,7059,31.867069,-90.443150
2162,MARION,MARION GENERAL HOSPITAL,"1560 SUMRALL RD,COLUMBIA,MARION,MS",MS,22462,31.255953,-89.805607


In [121]:
# No obvious pattern found
# Drop 2nd occurence of each duplicate
df.drop([2212,3947], inplace=True)

In [122]:
# Check rows have been dropped
df.loc[2211:2213]

,County Name,Facility_Name,Full_Address,State,Total,latitudes,longitudes
2211,TIPPAH,TIPPAH COUNTY HOSPITAL,"1005 CITY AVE NORTH,RIPLEY,TIPPAH,MS",MS,193,34.739039,-88.950607
2213,RANKIN,BRENTWOOD BEHAVIORAL HEALTHCARE OF MS,"3531 EAST LAKELAND DRIVE,FLOWOOD,RANKIN,MS",MS,1459,32.357990,-90.011870


In [123]:
df.loc[3945:3948]
# Notice that the index and iloc do not match anymore so we had to use loc
# and now have to reset index

,County Name,Facility_Name,Full_Address,State,Total,latitudes,longitudes
3945,HAMILTON,HAMILTON GENERAL HOSPITAL,"400 N BROWN, BUILDING 1,HAMILTON,HAMILTON,TX",TX,15125,31.703875,-98.113448
3946,HOUSTON,CROCKETT MEDICAL CENTER,"1100 E LOOP 304,CROCKETT,HOUSTON,TX",TX,2427,31.322152,-95.437114
3948,TARRANT,COOK CHILDRENS MEDICAL CENTER,"801 SEVENTH AVENUE,FORT WORTH,TARRANT,TX",TX,21617,30.851827,-95.187186


In [124]:
# Reset index
df.reset_index(drop=True, inplace=True)

# Check index has been reset
df.iloc[3945:3948]

,County Name,Facility_Name,Full_Address,State,Total,latitudes,longitudes
3945,KERR,KERRVILLE STATE HOSP,"721 THOMPSON DR,KERRVILLE,KERR,TX",TX,255,30.045887,-99.157744
3946,WICHITA,RED RIVER HOSPITAL,"1505 8TH ST,WICHITA FALLS,WICHITA,TX",TX,709,33.906316,-98.501335
3947,HARRIS,WEST OAKS HOSPITAL,"6500 HORNWOOD,HOUSTON,HARRIS,TX",TX,56280,29.708962,-95.496980


### 1.1.3 Reduce Memory Usage
Change data types of columns

In [125]:
# Inspect the data types of each column
df.dtypes

County Name       object
Facility_Name     object
Full_Address      object
State             object
Total              int64
latitudes        float64
longitudes       float64
dtype: object

In [126]:
# Find memory usage of each column
original_mem = df.memory_usage(deep=True)
original_mem

Index               128
County Name      286237
Facility_Name    381694
Full_Address     432295
State            263022
Total             35664
latitudes         35664
longitudes        35664
dtype: int64

In [127]:
# We know from our examination above that State only has 50 unique values
# It can be changed to a categorical variable
# Can anything else? Find the percentage of unique values in each category
df.select_dtypes(include=['object']).nunique()/df.select_dtypes(include=['object']).count()

County Name      0.285554
Facility_Name    0.969717
Full_Address     1.000000
State            0.011216
dtype: float64

In [128]:
# We see that County Name has about 29% unique values,
# Facility has 97%, Full_Address has 100%, and State has 1%
# The best candidate for categorical is the State column
df['State'] = df['State'].astype('category')
df.dtypes

County Name        object
Facility_Name      object
Full_Address       object
State            category
Total               int64
latitudes         float64
longitudes        float64
dtype: object

In [130]:
# Let's try out County Name to see if it's worth it
df['County Name'] = df['County Name'].astype('category')
df.dtypes

County Name      category
Facility_Name      object
Full_Address       object
State            category
Total               int64
latitudes         float64
longitudes        float64
dtype: object

In [131]:
# compute new memory usage
new_mem = df.memory_usage(deep=True)
new_mem

Index               128
County Name      131453
Facility_Name    381694
Full_Address     432295
State              9968
Total             35664
latitudes         35664
longitudes        35664
dtype: int64

In [132]:
# Compare original with updated memory usage
new_mem/original_mem

Index            1.000000
County Name      0.459245
Facility_Name    1.000000
Full_Address     1.000000
State            0.037898
Total            1.000000
latitudes        1.000000
longitudes       1.000000
dtype: float64

State has shrunk to 3.8% of it's original size.<br>
County Name has shrunk to 46% of it's original size

### 1.1.4 Import and Examine County Shapefile

Index            1.000000
County Name      0.459245
Facility_Name    1.000000
Full_Address     1.000000
State            0.037898
Total            1.000000
latitudes        1.000000
longitudes       1.000000
dtype: float64